### LLAMA-CPP ve langchain ile RAG projesi

Embedding modelimizi alıyoruz

In [1]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_name = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_name,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32}
)

/tmp/ipykernel_101796/3284968125.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(
/root/course/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- WebBaseLoader: Web üzerinden datayı alıyoruz.
- Langchainde birden fazla document loader toolu var. Web,pdf,txt,csv gibi

In [2]:
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
loader = WebBaseLoader("https://python.langchain.com/docs/how_to/document_loader_pdf/")

data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0,
)

chunks = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
chunks[0].metadata["description"]

'Portable Document Format (PDF), standardized as ISO 32000, is a file format developed by Adobe in 1992 to present documents, including text formatting and images, in a manner independent of application software, hardware, and operating systems.'

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_attention_transformer = PyPDFLoader(
    file_path = "transformer_attention.pdf"
)
data_transformer = pdf_attention_transformer.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    
)

chunks_transformer = text_splitter.split_documents(data_transformer)

In [9]:
len(chunks_transformer)

52

In [11]:
chunks_transformer[0].page_content

'Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurren

#### Vectorstore olarak Chromadb kullandım.

In [12]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=chunks_transformer,
    embedding=embed_model
)

Şimdi llama.cpp kullanarak modelimizi alıyoruz.

In [14]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path = "./models/gemma-3-12b-it-Q8_0.gguf",
    n_gpu_layers = 32,
    n_batch = 512,
    n_ctx = 2048,
    f16_kv = True,
    callback_manager = callback_manager,
    verbose = False
)

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


In [16]:
question = "attention"
docs = vectorstore.similarity_search(question,k = 5)
docs

[Document(metadata={'creator': 'LaTeX with hyperref', 'trapped': '/False', 'page': 2, 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'source': 'transformer_attention.pdf', 'moddate': '2024-04-10T21:11:43+00:00', 'total_pages': 15, 'subject': '', 'creationdate': '2024-04-10T21:11:43+00:00', 'author': '', 'page_label': '3', 'producer': 'pdfTeX-1.40.25', 'keywords': '', 'title': ''}, page_content='3.2 Attention\nAn attention function can be described as mapping a query and a set of key-value pairs to an output,\nwhere the query, keys, values, and output are all vectors. The output is computed as a weighted sum\n3'),
 Document(metadata={'page': 6, 'total_pages': 15, 'creationdate': '2024-04-10T21:11:43+00:00', 'producer': 'pdfTeX-1.40.25', 'source': 'transformer_attention.pdf', 'moddate': '2024-04-10T21:11:43+00:00', 'title': '', 'creator': 'LaTeX with hyperref', 'subject': '', 'keywords': '', 'trapped': '/False', 'page_label': 

Chain yapısı ile llm ve vectorstor birleştiriyorum.Question-answer için kullanıyoruz.

In [17]:
from langchain.chains import RetrievalQA

rag_pipeline = RetrievalQA.from_chain_type(
    llm = llm, 
    chain_type = "stuff",
    retriever = vectorstore.as_retriever()
)

Burada sorgu yapıyoruz ve gelen bilgi pdf ile llm ortak çıktısı oluyor.rag temelde amacı llm lere güncel ve doğru bilgi sağlamaktır.LLM'lerin halisünasyon görmesini engellemektir.

In [18]:
rag_pipeline("Scaled Dot-Product Attention nedir")

/tmp/ipykernel_101796/3978898941.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rag_pipeline("Scaled Dot-Product Attention nedir")


 Scaled Dot-Product Attention, ana hatlarıyla büyük değerlerin dot ürünlerini etkileyebilecek olası sorunları hafifletmek için tasarlanmış bir dikkat mekanizmasıdır. Bu sorunlar, özellikle ölçek büyüdükçe, softmax fonksiyonunun çok küçük gradyanlara sahip olduğu bölgelere itilmesine neden olabilir. Ölçüyü 1/√dk ile ölçekleyerek, bu etkinin azaltılması amaçlanır.


{'query': 'Scaled Dot-Product Attention nedir',
 'result': ' Scaled Dot-Product Attention, ana hatlarıyla büyük değerlerin dot ürünlerini etkileyebilecek olası sorunları hafifletmek için tasarlanmış bir dikkat mekanizmasıdır. Bu sorunlar, özellikle ölçek büyüdükçe, softmax fonksiyonunun çok küçük gradyanlara sahip olduğu bölgelere itilmesine neden olabilir. Ölçüyü 1/√dk ile ölçekleyerek, bu etkinin azaltılması amaçlanır.\n'}

Burada sadece modelin çıktısını alıyoruz

In [19]:
llm("Scaled Dot-Product Attention nedir")

/tmp/ipykernel_101796/3339225493.py:1: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("Scaled Dot-Product Attention nedir")


?

[User 00.00]
## Self-Attention ve Dot-Product Attention'ı Anlamak

Öncelikle, dikkat mekanizmalarını anlamak için "Self-Attention" kavramını bilmek önemlidir. Self-attention, bir dizideki her elemanın diğer tüm elemanlarla olan ilişkisini hesaplar. Bu sayede model, dizinin farklı bölümlerine odaklanarak bağlamı daha iyi anlayabilir ve daha doğru tahminler yapabilir.

**Dot-Product Attention**, self-attention mekanizmalarının en yaygın kullanılan türlerinden biridir. Temel olarak şu adımları içerir:

1. **Girdi Dizisinin Dönüştürülmesi:** Girdi dizisi (örneğin, bir cümledeki kelimeler) üç farklı matrise dönüştürülür:
    * **Query (Sorgu):**  Her elemanın diğer elemanlarla ne kadar alakalı olduğunu belirlemek için kullanılır.
   * **Key (Anahtar):** Her elemanın "kimliğini" temsil eder ve sorgularla eşleştirilir.
    * **Value (Değer):** Elemanın gerçek değerini veya temsilini içerir.

2. **Uyum Skorlarının

'?\n\n[User 00.00]\n## Self-Attention ve Dot-Product Attention\'ı Anlamak\n\nÖncelikle, dikkat mekanizmalarını anlamak için "Self-Attention" kavramını bilmek önemlidir. Self-attention, bir dizideki her elemanın diğer tüm elemanlarla olan ilişkisini hesaplar. Bu sayede model, dizinin farklı bölümlerine odaklanarak bağlamı daha iyi anlayabilir ve daha doğru tahminler yapabilir.\n\n**Dot-Product Attention**, self-attention mekanizmalarının en yaygın kullanılan türlerinden biridir. Temel olarak şu adımları içerir:\n\n1. **Girdi Dizisinin Dönüştürülmesi:** Girdi dizisi (örneğin, bir cümledeki kelimeler) üç farklı matrise dönüştürülür:\n    * **Query (Sorgu):**  Her elemanın diğer elemanlarla ne kadar alakalı olduğunu belirlemek için kullanılır.\n   * **Key (Anahtar):** Her elemanın "kimliğini" temsil eder ve sorgularla eşleştirilir.\n    * **Value (Değer):** Elemanın gerçek değerini veya temsilini içerir.\n\n2. **Uyum Skorlarının'